### 1. 개요
<img src = ./images/data-original.png width="600px">

**Brazilian E-Commerce Public Dataset by Olist**

**데이터 설명**

- 출처: [[캐글 링크]](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)
- 브라질 이커머스사 [Olist Store](https://olist.com/pt-br/)의 주문 데이터(가명화)
- 2016년~ 2018년 사이의 약 10만건의 주문 건

### 1-1 데이터 스키마
<img src = "./images/화면 캡처 2023-05-30 141031.png" width = "600px">

- 총 8개의 테이블로 구성
- 리뷰 테이블과 위치 테이블은 분석 사용 하지 않음

### 1-2 프로젝트 주요 목표

1. 재구매, 코호트, RFM 분석 등 비즈니스 분석을 수행하여 현상 해석 및 인사이트 도출
2. 빅쿼리를 통한 ad-hoc 분석

In [3]:
!pip install google-cloud-bigquery
!pip install db-dtypes
!pip install plotly
!pip install nbformat
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import matplotlib.pyplot as plt

import glob
from google.cloud import bigquery
from google.oauth2 import service_account

pd.set_option("display.max_columns",999)
warnings.filterwarnings("ignore")

In [ ]:
# 파이썬 구글 빅쿼리 연동 코드
# json 파일

key_path = glob.glob("./*.json")[0]
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)